In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv('vgsales.csv')
df

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [43]:
# mengelompokan data
data = df['NA_Sales'] 

min_value = data.min()
max_value = data.max()
range_data = max_value - min_value

# Hitung jumlah kelas dengan rumus Sturges
n = len(data) 
num_classes = int(1 + 3.322 * np.log10(n))

#  Hitung panjang interval
class_width = range_data / num_classes

# Tentukan batas interval
bins = np.arange(min_value, max_value + class_width, class_width)

# Buat tabel distribusi frekuensi
labels = [f"{round(bins[i], 2)} - {round(bins[i+1], 2)}" for i in range(len(bins)-1)]
frequency_table = pd.cut(data, bins=bins, labels=labels, right=False).value_counts().sort_index().reset_index()
frequency_table.columns = ['Interval', 'Frekuensi']

In [44]:
# mencari rata-rata hitung

# Pisahkan batas bawah dan atas
frequency_table[['Batas_Bawah', 'Batas_Atas']] = frequency_table['Interval'].str.split(' - ', expand=True).astype(float)

frequency_table['Titik_Tengah'] = (frequency_table['Batas_Bawah'] + frequency_table['Batas_Atas']) / 2

# Hitung f * m
frequency_table['f_m'] = frequency_table['Frekuensi'] * frequency_table['Titik_Tengah']

# Hitung rata-rata hitung
total_fm = frequency_table['f_m'].sum()
total_f = frequency_table['Frekuensi'].sum()
mean = total_fm / total_f

print(f"Rata-rata NA_Sales (berkelompok): {mean:.2f}")

Rata-rata NA_Sales (berkelompok): 1.43


In [45]:
# mencari median

# hitung frekuensi kumulatif
frequency_table['Frekuensi Kumulatif'] = frequency_table['Frekuensi'].cumsum()

# total frekuensi
n = frequency_table['Frekuensi'].sum()

# menentukan kelas median
median_class_idx = frequency_table[frequency_table['Frekuensi Kumulatif'] >= n / 2].index[0]

# hitung parameter
l_m = frequency_table.at[median_class_idx, 'Batas_Bawah'] # tepi bawah kelas median
f_m = frequency_table.at[median_class_idx, 'Frekuensi'] # frekuensi kelas median
c = frequency_table.at[median_class_idx, 'Batas_Atas'] - l_m # panjang kelas interval
F = 0 if median_class_idx == 0 else frequency_table.at[median_class_idx - 1, 'Frekuensi_Kumulatif'] # frekuensi kumulatif sebelum kelas median

# hitung median 
median = l_m + ((n / 2 - F) / f_m) * c

print(f"Median NA_Sales (berkelompok): {median:.2f}")

Median NA_Sales (berkelompok): 1.40


In [46]:
# mencari modus

# menentukan kelas modus (kelas dengan frekuensi tertinggi)
modus_class_idx = frequency_table['Frekuensi'].idxmax()

# hitung parameter modus
l_mo = frequency_table.at[modus_class_idx, 'Batas_Bawah'] # tepi bawah kelas modus
c = frequency_table.at[modus_class_idx, 'Batas_Atas'] # interval kelas
f_modus = frequency_table.at[modus_class_idx, 'Frekuensi'] # frekuensi kelas modus

# d1: frekuensi modus - frekuensi kelas sebelum modus
d1 = f_modus - (frequency_table.at[modus_class_idx - 1, 'Frekuensi'] if modus_class_idx > 0 else 0) 

# d2: frekuensi modus - frekuensi kelas setelah modus
d2 = f_modus - (frequency_table.at[modus_class_idx + 1, 'Frekuensi'] if modus_class_idx < len(frequency_table) - 1 else 0)

# hitung modus 
modus = l_mo + (d1 / (d1 + d2)) * c

print(f"Modus data kelompok: {modus:.2f}")

Modus data kelompok: 1.39


In [47]:
def hitung_kuartil(kelas, frekuensi, kuartil_ke):
    total_frekuensi = sum(frekuensi)  # Total frekuensi
    posisi_kuartil = kuartil_ke * total_frekuensi / 4  # Posisi kuartil (Q1, Q2, Q3)

    frekuensi_kumulatif = 0
    for indeks, frekuensi_kelas in enumerate(frekuensi):
        frekuensi_kumulatif += frekuensi_kelas
        if frekuensi_kumulatif >= posisi_kuartil:
            tepi_bawah = float(kelas[indeks].split(' - ')[0])  # Tepi bawah kelas kuartil
            panjang_interval = float(kelas[indeks].split(' - ')[1]) - tepi_bawah  # Panjang interval kelas
            frekuensi_kuartil = frekuensi_kelas  # Frekuensi kelas kuartil
            frekuensi_sebelum = frekuensi_kumulatif - frekuensi_kelas  # Frekuensi sebelum kelas kuartil

            # Hitung nilai kuartil
            kuartil = tepi_bawah + ((posisi_kuartil - frekuensi_sebelum) / frekuensi_kuartil) * panjang_interval
            return kuartil

# Pisahkan kelas dan frekuensi
kelas = frequency_table['Interval'].values.tolist()
frekuensi = frequency_table['Frekuensi'].values.tolist()

# Hitung kuartil
q1 = hitung_kuartil(kelas, frekuensi, 1)  # Kuartil ke-1
q2 = hitung_kuartil(kelas, frekuensi, 2)  # Kuartil ke-2 (median)
q3 = hitung_kuartil(kelas, frekuensi, 3)  # Kuartil ke-3

# Output
print(f"Kuartil ke-1 (Q1): {q1}")
print(f"Kuartil ke-2 (Q2): {q2}")
print(f"Kuartil ke-3 (Q3): {q3}")

Kuartil ke-1 (Q1): 0.6996665550617884
Kuartil ke-2 (Q2): 1.3993331101235769
Kuartil ke-3 (Q3): 2.0989996651853655


In [48]:
def hitung_desil(kelas, frekuensi, desil_ke):
    total_frekuensi = sum(frekuensi)  # Total frekuensi
    posisi_desil = desil_ke * total_frekuensi / 10  # Posisi desil (D1, D2, ..., D3)

    frekuensi_kumulatif = 0
    for indeks, frekuensi_kelas in enumerate(frekuensi):
        frekuensi_kumulatif += frekuensi_kelas
        if frekuensi_kumulatif >= posisi_desil:
            tepi_bawah = float(kelas[indeks].split(' - ')[0])  # Tepi bawah kelas desil
            panjang_interval = float(kelas[indeks].split(' - ')[1]) - tepi_bawah  # Panjang interval kelas
            frekuensi_desil = frekuensi_kelas  # Frekuensi kelas desil
            frekuensi_sebelum = frekuensi_kumulatif - frekuensi_kelas  # Frekuensi sebelum kelas desil

            # Hitung nilai desil
            desil = tepi_bawah + ((posisi_desil - frekuensi_sebelum) / frekuensi_desil) * panjang_interval
            return desil

# Pisahkan kelas dan frekuensi
kelas = frequency_table['Interval'].values.tolist()
frekuensi = frequency_table['Frekuensi'].values.tolist()

# Hitung desil
desil_values = {}
for i in range(1, 4):  # Desil D1 hingga D3
    desil_values[f"D{i}"] = hitung_desil(kelas, frekuensi, i)

# Output
for desil, value in desil_values.items():
    print(f"{desil}: {value}")

D1: 0.2798666220247154
D2: 0.5597332440494308
D3: 0.8395998660741463


In [49]:
total_frekuensi = frequency_table['Frekuensi'].sum()

# Fungsi untuk menghitung persentil
def hitung_persentil(kelas, frekuensi, persentil_ke):
    # Tentukan posisi persentil
    posisi_persentil = persentil_ke * (total_frekuensi + 1) / 100
    
    frekuensi_kumulatif = 0
    for i, f in enumerate(frekuensi):
        frekuensi_kumulatif += f
        
        # Pisahkan interval kelas menjadi tepi bawah dan tepi atas
        tepi_bawah, tepi_atas = map(float, kelas[i].split(' - '))
        
        if frekuensi_kumulatif >= posisi_persentil:  # Temukan kelas persentil
            panjang_interval = tepi_atas - tepi_bawah  # Panjang interval kelas
            f_p = f  # Frekuensi kelas persentil
            sigma_f = frekuensi_kumulatif - f  # Frekuensi kumulatif sebelum kelas persentil
            
            # Hitung persentil menggunakan rumus
            persentil = tepi_bawah + ((posisi_persentil - sigma_f) / f_p) * panjang_interval
            return persentil

# Tentukan kelas interval (dalam bentuk string yang berisi interval)
kelas = frequency_table['Interval'].values.tolist()

# Hitung persentil ke-25, ke-50 (median), dan ke-75
P25 = hitung_persentil(kelas, frequency_table['Frekuensi'].values.tolist(), 25)
P50 = hitung_persentil(kelas, frequency_table['Frekuensi'].values.tolist(), 50)
P75 = hitung_persentil(kelas, frequency_table['Frekuensi'].values.tolist(), 75)

# Output persentil
print(f"Persentil ke-25 (P25): {P25}")
print(f"Persentil ke-50 (P50): {P50}")
print(f"Persentil ke-75 (P75): {P75}")


Persentil ke-25 (P25): 0.6997087112680344
Persentil ke-50 (P50): 1.3994174225360687
Persentil ke-75 (P75): 2.099126133804103
